In [ ]:
import numpy as np
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt

In [ ]:
dataset=pd.read_csv('/content/Obesity Classification.csv')
dataset

In [ ]:

#checking for NAN values in dataset
dataset.isna().sum()

In [ ]:
#encoding verbal data
from sklearn.preprocessing import LabelEncoder
lex= LabelEncoder()
ley=LabelEncoder()
dataset.Gender=lex.fit_transform(dataset.Gender)
dataset.Gender


In [ ]:

dataset.Label=ley.fit_transform(dataset.Label)
dataset.Label

In [ ]:
X=dataset.iloc[:,:-1].values
Y=dataset.iloc[:,-1].values

In [ ]:
#hyperparameter-Tuning using gridsearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
model_params={
    'svm':{
        'model':svm.SVC(gamma='auto',probability=True),
        'params':{
            'C':[1,10,100,1000],
            'kernel':['rbf','linear']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    }
}

In [ ]:
#creating train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
#selecting best parameters in each model
scores=[]
best_estimators={}
for algo,mp in model_params.items():
 
  clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
  clf.fit(X_train,Y_train)
  scores.append({
      'model':algo,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_
  })
  best_estimators[algo]=clf.best_estimator_

df =pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
best_estimators['random_forest'].score(X_test,Y_test)

In [ ]:
best_estimators['svm'].score(X_test,Y_test)

In [ ]:
#selecting best classifier
best_clf=best_estimators['random_forest']

In [ ]:
#saving trained model to device
import joblib
joblib.dump(best_clf,'Trained_model.pkl')